In [9]:
import pandas as pd
import ast
import json as JSON

## Our Dataset

In [5]:
df = pd.read_csv('keystrokes-recipes.csv')
df.head(10)

,event_date,user_id,ks,recipe
0,2022-09-04 00:53:12.086991,5e68d82dd39ce517eaccd0c2,"[{'time': 1662252404346, 'character': 'Shift'}...","Firstly, cut up some chicken breasts into cube..."
1,2022-09-04 00:55:35.237120,5e87376eb4921a37c33affb4,"[{'time': 1662252644345, 'character': '1'}, {'...","First dice two onions, cut 2 tomatoes and 3 ch..."
2,2022-09-04 00:56:24.986568,5e68d82dd39ce517eaccd0c2,"[{'time': 1662252808442, 'character': 'Enter'}...",Title: Coconut and Tomato Curry\n\n\n\nIngredi...
3,2022-09-04 00:58:04.594495,5e87376eb4921a37c33affb4,"[{'time': 1662252958637, 'character': '1'}, {'...",Ingredients\n- 2 medium sized Onions\n- 2 medi...
4,2022-09-04 01:03:46.795075,6303814cf442fb34eaa1d118,"[{'time': 1662252871554, 'character': 'I'}, {'...",Ingredients\n- potato\n- carrots\n- parsnips\n...
5,2022-09-04 01:06:06.441679,5e87376eb4921a37c33affb4,"[{'time': 1662253173760, 'character': 'CapsLoc...",Victoria sponge cake\nIngredients\n- 250 grams...
6,2022-09-04 01:06:26.244137,6303814cf442fb34eaa1d118,"[{'time': 1662253570675, 'character': 'Enter'}...",Lamb Leg and Vegetable\n\nIngredients\n- potat...
7,2022-09-04 01:06:30.884313,5e87376eb4921a37c33affb4,"[{'time': 1662253586831, 'character': ' '}, {'...",Victoria sponge cake\nIngredients\n- 250 grams...
8,2022-09-04 01:06:37.199980,5e68d82dd39ce517eaccd0c2,"[{'time': 1662253176308, 'character': 'I'}, {'...",Title: Chicken Risotto\n\n\n\nIngredients:\n- ...
9,2022-09-04 01:08:05.510372,5e68d82dd39ce517eaccd0c2,"[{'time': 1662253611905, 'character': 'Enter'}...",Title: Chicken Risotto\n\n\n\nIngredients:\n- ...


#### Description

The platform the users used for writing recipes is a ML based review on recipes users write. The platform is called __RELEX__ and was designed to study the behavior of users. In fact, the users were tasked with writing three recipes each and they were all divided into 4 groups. 
| Group 1 | Group 2| Group 3 | Group 4|
| :----: | :---:|:----:|:---:|
Without Reflective Prompts | With Reflective Prompts | Without Reflective Prompts | With Reflective Prompts
Without Adaptive Feedback | Without Adaptive Feedback | With Adaptive Feedback  |With Adaptive Feedback

It's important to note some users wrote less than 3 and others wrote more than 3 (at least from what i've seen in the data)


We have 187 users (each represented by a ```user_id```) and 1091 sets of keystrokes.
Each set of keystrokes is a users revision on what they wrote previously or the start of a new recipe. What I imagine a scenario is when using the platform is 
- User x writes a first version of a recipe, clicks finished button (registers as a set of keystrokes in our dataset)
- According to his group, the platform may or may not present some suggestions
- The user modifies their text and submits again (registers as a second set of keystrokes in our dataset)
- If the user is done, starts the second recipe, else revises again and so on.


In the cell below we define keywords as characters appearing in the dataset that correspond to a keyboard action. We also define ```noisy_punct``` as noisy ponctuation characters that we want to remove from certain analysis we will make. We also copy the original data to a new file we will later modify.

In [6]:
KEYWORDS = ['ArrowDown', 'ArrowLeft', 'ArrowRight', 'ArrowUp', 'Backspace', 'CapsLock', 'Control', 'Delete', 'End', 'Enter', 'Shift']
noisy_punct = [',', '.', '-', ':', '(', ')']
#create a copy of the dataset to another csv file
csv_filename = 'keystrokes-recipes-modified.csv'
df.to_csv(csv_filename, index=False)

- ```keystrokes-recipes.csv``` is the original data and we keep it in case we want to look back at one moment
- ```keystrokes-recipes-modified.csv``` is the modified data


## Data cleaning and sorting

Our data consists of a csv file with event dates, user ids, keystrokes and the recipes they wrote.
We clean all the data by working throught the keystrokes first.

* We group the characters into the word written and separate between important keywords typed such as backspace, shift, enter etc. The sequence ['shift', 'p', 'e', 'r'] becomes ['shift', 'per'] 
* We sort the data by user id then event date to get a better idea of every recipe every student has written and the time they took.

### Processing the data

The first thing we did was isolate the keystrokes to a new ```json``` file saved in ```data/ks.json```

The next step is to group words together and separate them from keywords and we work between each whitespace.
 
So for example this entry: 
```{'time': 1662252404346, 'character': 'Shift'}, {'time': 1662252404376, 'character': 'f'}, {'time': 1662252404505, 'character': 'i'}, {'time': 16622524046700, 'character': ' '}``` 

gives the following output: 
```{'time': 1662252404346, 'word': 'Shift'}, {'time': 1662252404505, 'word': 'fi'}```


In [10]:
def find_seq(chars):
    return "".join(list(filter(lambda _ : _ not in KEYWORDS, chars)))

def separate_entry(json_values):
    new_data = []
    last_whitespace = 0
    characters = [arr[1] for arr in json_values]
    for i, (time, character) in enumerate(json_values):
        if character.isspace():
            word = characters[last_whitespace: i]
            if not any(i in word for i in KEYWORDS):
                new_data.append({'time': time, 'word': "".join(word)})
            else:
                new_data.append({'time': time, 'word': find_seq(word)})
            last_whitespace = i+1
        elif character in KEYWORDS:
            new_data.append({'time': time, 'word': character})
        elif not character:
            continue
    return new_data

new_df = pd.DataFrame(pd.read_json("data/ks.json"))
arr = []
for jsonf in new_df.values:
    sub_arr = []
    for d in jsonf:
        if d is not None:
            sub_arr.append([d["time"], d["character"]])
    arr.append(sub_arr)

result = []
for jsonf in arr:
    result.append(separate_entry(jsonf))
with open("data/new_data.json", "w") as f:
    JSON.dump(result, f)

Basically, we will format the data for the ```separate_entry``` function and when everything is computed, it dumps all the data in a new json file: ```new_data.json``` in the ```data``` directory.

```separate_entry``` computes the words between each space character, all the while separating words from keywords. It uses the function ```find_seq``` to separate the characters from keywords so it allows to isolate words between each whitespaces.

### Modifying the CSV file

We just modify the keystroke data for each row of the original data in ```keystrokes-recipes.csv``` but apply it to ```keystrokes-recipes-modified.csv```

In [11]:
jsons = pd.read_json('data/new_data.json').values.tolist()

for i, json in enumerate(jsons):
    jsons[i]= list(filter(lambda _ : _ is not None, json))

def write_to_csv_file(filename, recipes_len):
    dframe = pd.read_csv(filename)
    for i in range(recipes_len):
        dframe.iloc[i]['ks'] = jsons[i]
    dframe.to_csv(filename, index=False)

write_to_csv_file(csv_filename, len(jsons))

## Sorting by `user_id` and `event_date`

We first sort by user id in order to differentiate behaviour between different people more easily and then by event date

In [12]:
#we sort 
pd.read_csv(csv_filename).sort_values(by=['user_id', 'event_date'], ascending=True).to_csv(csv_filename, index=False)
#update the dataframe with which we work
df = pd.read_csv(csv_filename)
df.head(10)

,event_date,user_id,ks,recipe
0,2022-09-04 03:28:18.613319,55ae64defdf99b3f864653e7,"[{'time': 1662261900176, 'word': 'Shift'}, {'t...",Brown 1 pound of hamburger meat. Drain the gre...
1,2022-09-04 03:29:37.124556,55ae64defdf99b3f864653e7,"[{'time': 1662261900176, 'word': 'Shift'}, {'t...",1) Brown 1 pound of hamburger meat. Drain the ...
2,2022-09-04 03:29:47.816111,55ae64defdf99b3f864653e7,"[{'time': 1662261900176, 'word': 'Shift'}, {'t...",1) Brown 1 pound of hamburger meat. Drain the ...
3,2022-09-04 03:33:03.555075,55ae64defdf99b3f864653e7,"[{'time': 1662262224842, 'word': 'CapsLock'}, ...","1) Cook chicken as desired (boiled, pan seared..."
4,2022-09-04 03:33:30.062465,55ae64defdf99b3f864653e7,"[{'time': 1662262224842, 'word': 'CapsLock'}, ...","1) Cook chicken as desired (boiled, pan seared..."
5,2022-09-04 03:34:08.666681,55ae64defdf99b3f864653e7,"[{'time': 1662262224842, 'word': 'CapsLock'}, ...","1) Cook chicken as desired (boiled, pan seared..."
6,2022-09-04 03:35:33.869167,55ae64defdf99b3f864653e7,"[{'time': 1662262452660, 'word': '28'}, {'time...",28 oz or so of potatoes cubed\n1 8oz of cream ...
7,2022-09-04 03:36:08.454643,55ae64defdf99b3f864653e7,"[{'time': 1662262452660, 'word': '28'}, {'time...",28 oz or so of potatoes cubed\n1 8oz of cream ...
8,2022-09-04 03:36:43.142187,55ae64defdf99b3f864653e7,"[{'time': 1662262452660, 'word': '28'}, {'time...",28 oz or so of potatoes cubed\n1 8oz of cream ...
9,2022-09-04 14:01:31.746981,55d22025cc2b18000c0b9d9c,"[{'time': 1662298866838, 'word': 'Shift'}, {'t...",To serve 4 people (your family!)\n\n\n-You wil...


we edit the file ```new_data.json``` in order to have the keystrokes in the sorted fashion, by user and event date. This will prevent future issues. We also created the file ```keystrokes_sorted_by_user.json```

In [13]:
data = [ast.literal_eval(df['ks'].values[i]) for i in range(len(df))]
with open("data/new_data.json", "w") as f:
        JSON.dump(data, f)